In [163]:
import requests
from datetime import datetime

REQUEST_BATCH_SIZE=1000

In [164]:
def getGroupMembersFiltered(group_id,fields, *filter_functions):
    count=requests.get('https://api.vk.com/method/groups.getMembers?group_id='+group_id+'&fields='+str(fields)+'&version=5.27&access_token=bcabe0c4bcabe0c4bcabe0c4f5bcc38eefbbcabbcabe0c4e0fe7790b7e3f8cf5cefa92b').json()['response']['count']
    
    log( 'Members in group '+group_id+' : '+ str(count))
    
    requests_count=count//REQUEST_BATCH_SIZE + 1;

    all_filtered_members=list()    
    for i in range(requests_count):
        members_batch=requests.get('https://api.vk.com/method/groups.getMembers?group_id='+group_id+'&fields='+str(fields)+'&offset='+str(i*REQUEST_BATCH_SIZE)+'&version=5.27&access_token=bcabe0c4bcabe0c4bcabe0c4f5bcc38eefbbcabbcabe0c4e0fe7790b7e3f8cf5cefa92b').json()['response']['users']
        all_filtered_members.extend(fitlerListBy(members_batch,filter_functions))
        
    return all_filtered_members;

def fitlerListBy(list, filter_functions):
    return [x for x in list if (combineBoolFunctions(filter_functions,x))]

def combineBoolFunctions(functions,x):
    result=True
    for func in functions:
        result=result and func(x)
    return result

def writeToFile(file_name,items):
    with open(file_name, 'w') as f:
        for item in items:
            f.write("%s\n" % item)

In [165]:

def memberHasOpenMessage(member):
    return ('can_write_private_message' in member and member['can_write_private_message'] ==1)
    
def memberHasAppropriateChild(member):
    if('relatives' in member):
        for relative in member['relatives']:
            if('type' in relative and 'uid' in relative and relative['type']=='child' and relative['uid'] > 0):
                return isChildAppropriate(relative['uid'])
    return False

def isChildAppropriate(child_id):
    child_profile=requests.get("https://api.vk.com/method/users.get?user_id="+str(child_id)+"&fields=bdate,universities,education,occupation&version=5.27&access_token=bcabe0c4bcabe0c4bcabe0c4f5bcc38eefbbcabbcabe0c4e0fe7790b7e3f8cf5cefa92b").json()['response'][0]
    return childHasAppropriateAge(child_profile) and childIsSchoolar(child_profile)
    
def childHasAppropriateAge(child):
    if ('bdate' in child and child['bdate'].count('.') == 2):
        date = datetime.strptime(child['bdate'], "%d.%m.%Y")
        return (date.year >= 2000 and date.year <=2003)
            
    return False
    
def childIsSchoolar(child):
    has_not_univer = (('universities' not in child or child['universities'] == [] ) and 'university' not in child and 'university_name' not in child)
    by_occupation =  ('occupation' not in child or child['occupation']['type'] != 'university')
    return not has_not_univer and by_occupation;
    
def log(text):
    print(datetime.now().strftime("%Y-%m-%d %H:%M : ") + text)

In [ ]:
group_ids=['52388302',
'm_roditeli',
'roditeli_i',
'parents.club',
'roditeli20',
'glad_parents',
'modernparent',
'mother4you',
'club_razvivajka' ,
'vk.children',
'strana_detey',
'29746763',
'public.nash',
'mamas_and_papas',
'kidsandwe']
    
parents=list()

for group_id in group_ids:
    log('Start process group: '+group_id)
    parents.extend(getGroupMembersFiltered(group_id,'relatives,can_write_private_message', memberHasOpenMessage, memberHasAppropriateChild))
    log('Add group: '+group_id)

parents_links = list(map(lambda x: "https://vk.com/id" + str(x['uid']), parents))
writeToFile('parents.txt',parents_links)


2019-02-06 13:31 : Start process group: 52388302
2019-02-06 13:31 : Members in group 52388302 : 1294105
